In [1]:
import sys
sys.path.insert(0, '../')

import pandas as pd
from preprocess import Audio_Processor
from data_utils import balanced_supersample, balanced_subsample
from sklearn import metrics
from classification_plots import plot_confusion_matrix, plot_learning_curve
import matplotlib.pyplot as plt
import os
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import time
from collections import Counter

Using TensorFlow backend.


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Able to specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="1"

SR = 16000
blocksize = int(SR/2)
overlap = int(SR/4)

orig_SR = 44100
orig_blocksize = int(orig_SR * 5)
orig_overlap = 0 #int(orig_SR/4)

# Load Dataset
Here we load the csv that describes each file in the dataset. We add a high level category that is defined in the ESC-50 documentation. This we realize is anthetical to true training, it is a stopgap for when we use NLP to classify tags into these categories.

In [4]:
path_to_db='../../ESC-50/'
audio_dir = path_to_db + 'audio/'
ps = Audio_Processor(path_to_db + 'audio/', sr=SR)
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50
h_classes = ['Human & Animal', 'Interacting Materials']
mapping = {'dog': 0,'rooster': 0,'pig': 0,'cow': 0,'frog': 0,'cat': 0,'hen': 0,
            'insects': 0,'sheep': 0,'crow': 0,'rain': 1,'sea_waves': 1,'crackling_fire': 1,
            'crickets': 0,'chirping_birds': 0,'water_drops': 1,'wind': 1,'pouring_water': 1,
            'toilet_flush': 1,'thunderstorm': 1,'crying_baby': 0,'sneezing': 0,'clapping': 0,
            'breathing': 0,'coughing': 0,'footsteps': 1,'laughing': 0,'brushing_teeth': 1,
            'snoring': 0,'drinking_sipping': 1,'door_wood_knock': 1,'mouse_click': 1,
            'keyboard_typing': 1,'door_wood_creaks': 1,'can_opening': 1,'washing_machine': 1,
            'vacuum_cleaner': 1,'clock_alarm': 1,'clock_tick': 1,'glass_breaking':1,'helicopter': 1,
            'chainsaw': 1,'siren': 1,'car_horn': 1,'engine': 1,'train': 1,'church_bells': 1,
            'airplane': 1,'fireworks': 1,'hand_saw': 1,
            }
dataset['h_target'] = None
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']
    dataset.loc[index, 'h_target'] = mapping[row['category']]

In [ ]:
import time
import librosa
import numpy as np
import soundfile as sf

def load_audio_blockwise(data, blocksize=1024, overlap=512, debug=False):
    start_time = time.time()
    items = []
    target = []
    h_target = []
    for i, sample in data.iterrows():
        if debug:
            print("File Processing", end="", flush=True)
        blockgen = sf.blocks(audio_dir + sample['filename'], 
                             blocksize=blocksize, 
                             overlap=overlap, 
                             always_2d=True,
                             fill_value=0.0)
        sr = sf.info(audio_dir + sample['filename']).samplerate
        for bl in blockgen:
            if not np.any(bl):
                continue
            if debug:
                print(".", end="", flush=True)
            y = bl.transpose()
            y = librosa.resample(y, sr, SR)
            y = y[:int(blocksize)]
            y = y[np.newaxis, :]
            items.append(y)
            h_target.append(sample.h_target)
            target.append(sample.target)
        if debug:
            print("Done")
    print("\tProcessing Time: " + str(time.time() - start_time))
    return np.vstack(items), np.array(h_target), np.array(target)

def load_file_blockwise(filename, blocksize=1024, overlap=512, debug=False):
    items = []
    if debug:
        print("File Processing", end="", flush=True)
    blockgen = sf.blocks(audio_dir + filename, 
                         blocksize=blocksize, 
                         overlap=overlap, 
                         always_2d=True,
                         fill_value=0.0)
    sr = sf.info(audio_dir + filename).samplerate
    for bl in blockgen:
        if not np.any(bl):
            continue
        if debug:
            print(".", end="", flush=True)
        y = bl.transpose()
        y = librosa.resample(y, sr, SR)
        y = y[:int(blocksize)]
        y = y[np.newaxis, :]
        items.append(y)
        
    if debug:
        print("Done")

    return items

In [ ]:
full_X, _, _ = load_audio_blockwise(dataset, orig_blocksize, orig_overlap)
df = ps.preprocess_fold(dataset,
                        kind='mfcc',
                        blocksize=blocksize,
                        overlap=overlap,
                        feature_bag=False,
                        folds=5
                       )
df['fold'] = dataset.fold
df['full'] = [a for a in full_X.squeeze()]
df.columns = ['target', 'h_target', 'derived', 'fold', 'base']
df.head()

	Processing Time: 299.20084166526794


,target,h_target,derived,fold,base
0,0,0,mfcc_2_std mfcc_2_mean mfcc_2_noise mfcc...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,14,0,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,1,"[-0.042354584992188686, -0.12557755392933506, ..."
2,36,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,1,"[-0.0071510895789658885, -0.008539486532633997..."
3,36,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,1,"[0.3236247120233376, 0.23597562357962842, 0.49..."
4,19,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,1,"[-0.0004667657144710984, -0.000415740161992739..."


In [ ]:
train = df[df.fold != 1].reset_index(drop=True)
train.head()

,target,h_target,derived,fold,base
0,43,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,2,"[0.03132703551358152, 0.049165900621142844, 0...."
1,1,0,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,2,"[-3.7501595140290916e-05, -0.00010058510417064..."
2,10,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,2,"[-0.09333880784071064, -0.029133682284672446, ..."
3,17,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,2,"[-0.0002920302234656099, -0.000457424359913381..."
4,17,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,2,"[-0.0015238863621668303, -0.006384021990456582..."


In [ ]:
test = df[df.fold == 1].reset_index(drop=True)
test.head()

,target,h_target,derived,fold,base
0,0,0,mfcc_2_std mfcc_2_mean mfcc_2_noise mfcc...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,14,0,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,1,"[-0.042354584992188686, -0.12557755392933506, ..."
2,36,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,1,"[-0.0071510895789658885, -0.008539486532633997..."
3,36,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,1,"[0.3236247120233376, 0.23597562357962842, 0.49..."
4,19,1,mfcc_2_std mfcc_2_mean mfcc_2_noise mfc...,1,"[-0.0004667657144710984, -0.000415740161992739..."


In [ ]:
train_X = train.drop(['target', 'h_target', 'fold'], axis=1)
train_y = train[['h_target','target']]
test_X = test.drop(['target', 'h_target', 'fold'], axis=1)
test_y = test[['h_target','target']]

## Classifier Definition

In [ ]:
%%file mixd_mult.py

import numpy as np
import pandas as pd
import inspect
from sklearn.base import BaseEstimator, ClassifierMixin
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, InputLayer
from kapre.time_frequency import Melspectrogram, Spectrogram

from keras.wrappers.scikit_learn import KerasClassifier

SR=16000

class Mixed_Multilayer(BaseEstimator, ClassifierMixin):
    def __init__(self, epochs=50, batch_size=128, validation_split=0.05,
                       a_epochs=50, a_batch_size=128,
                       i_epochs=50, i_batch_size=128,
                       verbose=1, proc=None):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")

        for arg, val in values.items():
            setattr(self, arg, val)
            pass
    
    
    def fit(self, X, y):
        
        X_b = np.array([i for i in X['base']])
        X_d = pd.concat([j for j in X['derived']], ignore_index=True)
        
        y_p = []
        yy = []
        a_y = []
        i_y = []
        for i in range(0, len(X)):
            t_y = y.at[i, 'h_target']
            t_yy = y.at[i, 'target']
            length = len(X.at[i, 'derived'])
            yy += [t_yy] * length
            y_p += [t_y] * length
            if t_y == 0:
                a_y += [t_yy] * length
            else:
                i_y += [t_yy] * length
        
        if self.proc:
            X_d = self.proc.fit_transform(X_d, yy)
        
        dims = X_d.shape[1]
#         print(X_b.shape)
#         print(X_d.shape)
#         print(np.array(yy).shape)
#         print(np.array(y_p).shape)
        
        """Top layer of hierarchy"""
        self.clf = KerasClassifier(build_fn=self.deep_net,
                                   feature_count=SR * 5,
                                   epochs=self.epochs, 
                                   batch_size=self.batch_size, 
                                   validation_split=self.validation_split,
                                   verbose=self.verbose
                                  )
        self.clf.fit(X_b[:, np.newaxis, :], y.h_target)
        
        """Animal Layer"""
        self.a_clf = KerasClassifier(build_fn=self.deep_net_a,
                                       feature_count=dims,
                                       epochs=self.a_epochs, 
                                       batch_size=self.a_batch_size, 
                                       validation_split=self.validation_split,
                                       verbose=self.verbose
                                    )
        
        self.a_clf.fit(X_d[np.isin(y_p, 0)], a_y)
        
        """Interacting Materials Layer"""
        self.i_clf = KerasClassifier(build_fn=self.deep_net_i, 
                                       feature_count=dims,
                                       epochs=self.i_epochs, 
                                       batch_size=self.i_batch_size, 
                                       validation_split=self.validation_split,
                                       verbose=self.verbose
                                    )
        self.i_clf.fit(X_d[np.isin(y_p, 1)], i_y)
        
        return self
        
    def predict(self, X, y=None):
        predictions=[]

        X_b = np.array([i for i in X['base']])
        X_d = X['derived']
        
        prob = self.clf.predict_proba(X_b[:, np.newaxis, :], verbose=0)

        for i, d in enumerate(X_d):
#             print("Document: " + str(i))
            # Transform before estimation
            if self.proc:
                d = self.proc.transform(d)
            
            if prob[i,0] > 0.75:
                pred = self.a_clf.predict(d, verbose=0)
            elif prob[i,1] > 0.75:
                pred = self.i_clf.predict(d, verbose=0)
            else:
                a_prob = self.a_clf.predict_proba(d, verbose=0) * prob[i, 0]
                i_prob = self.i_clf.predict_proba(d, verbose=0) * prob[i, 1]

                a_prob = np.prod(a_prob, axis=0)
                i_prob = np.prod(i_prob, axis=0)
                
                if(np.max(a_prob) > np.max(i_prob)):
                    pred = self.a_clf.predict(d, verbose=0)
                else:
                    pred = self.i_clf.predict(d, verbose=0)

            pred = np.bincount(pred).argmax()
            predictions.append(pred)
        
        return predictions
    
    def predict_per_frame(self, X, y=None):
        predictions=[]

        X_b = np.array([i for i in X['base']])
        X_d = X['derived']
        
        prob = self.clf.predict_proba(X_b[:, np.newaxis, :], verbose=0)

        for i, d in enumerate(X_d):
#             print("Document: " + str(i))
            # Transform before estimation
            if self.proc:
                d = self.proc.transform(d)
            
            if prob[i,0] > 0.75:
                pred = self.a_clf.predict(d, verbose=0)
            elif prob[i,1] > 0.75:
                pred = self.i_clf.predict(d, verbose=0)
            else:
                a_prob = self.a_clf.predict_proba(d, verbose=0) * prob[i, 0]
                i_prob = self.i_clf.predict_proba(d, verbose=0) * prob[i, 1]

                a_prob = np.prod(a_prob, axis=0)
                i_prob = np.prod(i_prob, axis=0)
                
                if(np.max(a_prob) > np.max(i_prob)):
                    pred = self.a_clf.predict(d, verbose=0)
                else:
                    pred = self.i_clf.predict(d, verbose=0)

            predictions.append(pred)
        
        return predictions
    
    def predict_proba(self, X, y=None):
        
        X_b = np.array([i for i in X['base']])
        X_d = X['derived']
        
        prob = self.clf.predict_proba(X_b[:, np.newaxis, :], verbose=0)
        
        for i, d in enumerate(X_d):
            if self.proc:
                X_d = self.proc.transform(d)
                
            prob_a = np.multiply(np.prod(self.a_clf.predict_proba(X_d, verbose=0), axis=0),prob[i,0])
#             print(prob_a.shape)

            prob_i = np.multiply(np.prod(self.i_clf.predict_proba(X_d, verbose=0), axis=0),prob[i,0])
#             print(prob_i.shape)
#             print()
        
        probs = [None] * 50
        for counter, j in enumerate(self.a_clf.classes_):
            probs[j] = prob_a[counter]
        for counter, j in enumerate(self.i_clf.classes_):
            probs[j] = prob_i[counter]
            
        return np.array(probs)
        
    
    def deep_net(self, feature_count):
        # Create Model
        model = Sequential()
        model.add(Melspectrogram(
            sr=SR,
            n_mels=128,
            power_melgram=1.0,
            input_shape=(1, feature_count),
            trainable_fb=False,
            fmin = 800,
            fmax = 8000
        ))
        model.add(Convolution2D(32, 9, 9, name='conv1', activation='relu'))
        model.add(MaxPooling2D((25, 17)))
        model.add(Flatten())
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

        # Compile model
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        model.summary()


        return model

    def deep_net_a(self, feature_count):
        model = Sequential()
        model.add(Dense(64, activation='relu', input_shape=(50,)))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(18, kernel_initializer='normal', activation='softmax'))

        # Compile model
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        model.summary()


        return model

    def deep_net_i(self, feature_count):
        model = Sequential()
        model.add(Dense(64, activation='relu', input_shape=(50,)))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(32, kernel_initializer='normal', activation='softmax'))

        # Compile model
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        model.summary()


        return model

Overwriting mixd_mult.py


In [ ]:
from mixd_mult import Mixed_Multilayer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.decomposition import PCA

proc = Pipeline([
    ('scaler', MinMaxScaler()),
    ('feat_sel', SelectKBest(k=80, score_func=chi2)),
    ('feat_red', PCA(n_components=50))
])

clf = Mixed_Multilayer(proc=proc)

clf.fit(train_X, train_y)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_1 (Melspectro (None, 128, 313, 1)       296064    
_________________________________________________________________
conv1 (Conv2D)               (None, 120, 305, 32)      2624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 17, 32)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2176)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                69664     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
__________

In [ ]:
pred = clf.predict(test_X)
print(metrics.accuracy_score(test_y.target, pred))
print(metrics.precision_score(test_y.target, pred, average='macro'))
cm = metrics.confusion_matrix(test_y.target, pred)
plt.figure(figsize=(20,20))
plot_confusion_matrix(cm, classes)
plt.show()

In [ ]:
# %%time
import time
from load_blockwise import load_file_blockwise

predictions = []
full_targets = []
start_time = time.time()
top_pred_probs = []
pred_probs = []

for data_iloc in range(0,len(test)):
    s_time = time.time()

    X = test.iloc[data_iloc][['derived', 'base']]
    y = test.iloc[data_iloc]['target']

#     Made for batch processing usually
    X['base'] = X['base'][np.newaxis, :]
    X['derived'] = [X['derived']]
    
    pred = clf.predict(X)
    predictions.append(pred)
    
    full_targets.append(y)
    print("\tFile Time: " + str(time.time() - s_time))

print("\tProcessing Time: " + str(time.time() - start_time))

In [ ]:
print(metrics.accuracy_score(full_targets, predictions))
print(metrics.precision_score(full_targets, predictions, average='macro'))
cm = metrics.confusion_matrix(full_targets, predictions)
plt.figure(figsize=(20,20))
plot_confusion_matrix(cm, classes)
plt.show()

In [ ]:
def query_dataset(query_term):
    h_l = mapping[query_term]
    l_l = classes.index(query_term)
    
    predictions = []

    for data_iloc in range(0,len(test)):
        X = test.iloc[data_iloc][['derived', 'base']]
        y = test.iloc[data_iloc]['target']

    #     Made for batch processing usually
        X['base'] = X['base'][np.newaxis, :]
        X['derived'] = [X['derived']]

        prob = clf.predict_proba(X)[l_l]
        pred = clf.predict_per_frame(X)
        
        predictions.append({
            'ds_id': data_iloc,
            'prob': prob,
            'prediction': pred
        })
        
    predictions = pd.DataFrame(predictions).sort_values(by=['prob'], ascending=False).reset_index(drop=True)
    return predictions

In [ ]:
%%time
preds = query_dataset('dog')

In [ ]:
preds.head()

In [24]:
# Checking index
def check_accuracy(preds, query, count):
    misclass = []
    
    print(query)
    for data_iloc in range(0, count):
        row = preds.iloc[data_iloc]
        if dataset.iloc[row.ds_id].target != classes.index(query):
            misclass.append(classes[dataset.iloc[row.ds_id].target])
            
    print(misclass)
    print()
    return (count - len(misclass))/count

In [27]:
# Checking index
check_accuracy(preds, 'dog', 20)

dog
['hand_saw', 'glass_breaking', 'hand_saw', 'hand_saw', 'breathing', 'car_horn', 'coughing', 'hand_saw', 'sneezing', 'breathing', 'breathing', 'coughing', 'glass_breaking', 'car_horn', 'clock_tick', 'glass_breaking', 'rooster', 'hand_saw', 'rooster']



0.05

In [28]:
accuracies = []
for cls in classes:
    preds = query_dataset(cls)
    acc = check_accuracy(preds, cls, 20)
    print(acc)
    accuracies.append(acc)
    
np.average(accuracies)

dog
['cow', 'glass_breaking', 'door_wood_knock', 'coughing', 'coughing', 'car_horn', 'mouse_click', 'sneezing', 'coughing', 'footsteps', 'drinking_sipping', 'glass_breaking', 'door_wood_knock', 'sneezing', 'insects']

0.25
rooster
['coughing', 'dog', 'car_horn', 'insects', 'sneezing', 'car_horn', 'glass_breaking', 'coughing', 'glass_breaking', 'sneezing', 'water_drops', 'coughing', 'door_wood_knock', 'dog', 'can_opening', 'footsteps', 'door_wood_knock', 'glass_breaking']

0.1
pig
['glass_breaking', 'car_horn', 'breathing', 'breathing', 'hand_saw', 'breathing', 'coughing', 'hand_saw', 'glass_breaking', 'hand_saw', 'car_horn', 'brushing_teeth', 'sneezing', 'rooster', 'coughing', 'dog', 'brushing_teeth', 'breathing', 'clock_tick']

0.05
cow
['car_horn', 'sneezing', 'glass_breaking', 'dog', 'coughing', 'insects', 'glass_breaking', 'coughing', 'breathing', 'water_drops', 'rooster', 'dog', 'car_horn', 'footsteps', 'laughing', 'breathing', 'glass_breaking', 'sneezing']

0.1
frog
['clock_alarm

washing_machine
['car_horn', 'dog', 'footsteps', 'glass_breaking', 'sneezing', 'coughing', 'dog', 'glass_breaking', 'crickets', 'breathing', 'wind', 'coughing', 'rooster', 'crickets', 'wind', 'rooster', 'engine', 'breathing']

0.1
vacuum_cleaner
['glass_breaking', 'breathing', 'breathing', 'breathing', 'crickets', 'car_horn', 'car_horn', 'washing_machine', 'breathing', 'sneezing', 'crickets', 'rooster', 'wind', 'wind', 'coughing', 'wind']

0.2
clock_alarm
['glass_breaking', 'coughing', 'breathing', 'breathing', 'sneezing', 'hand_saw', 'car_horn', 'glass_breaking', 'breathing', 'coughing', 'dog', 'glass_breaking', 'glass_breaking', 'car_horn', 'rooster', 'car_horn', 'pouring_water', 'laughing', 'footsteps']

0.05
clock_tick
['dog', 'car_horn', 'coughing', 'footsteps', 'dog', 'glass_breaking', 'coughing', 'insects', 'coughing', 'sneezing', 'sneezing', 'water_drops', 'can_opening', 'door_wood_knock', 'door_wood_knock', 'dog', 'mouse_click', 'mouse_click']

0.1
glass_breaking
['sneezing', 

0.12900000000000003

## High-Level Shallow Nets
Train binary shallow nets for high level categories(animals, natural, human, domestic, urban)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, InputLayer
import keras.metrics as kmet
from kapre.time_frequency import Melspectrogram, Spectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise

ensemble_num = 10
num_hidden_neurons = 10
dropout = 0.25

epochs = 50
batch = 128

def deep_net():
    # Create Model
    model = Sequential()
    model.add(Melspectrogram(
        sr=SR,
        n_mels=128,
        power_melgram=1.0,
        input_shape=(1, SR * 5),
        trainable_fb=False,
        fmin = 800,
        fmax = 8000
    ))
    model.add(Convolution2D(32, 9, 9, name='conv1', activation='relu'))
    model.add(MaxPooling2D((25, 17)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', kmet.mae])
    
    model.summary()

    
    return model

In [ ]:
clf = Pipeline([
    ('classify', KerasClassifier(build_fn=deep_net, 
                       epochs=epochs, 
                       batch_size=batch, 
                       validation_split=0.15)
    )
])

clf.fit(train_X, train_y)

# Low-Level Deep Net

### Animal

In [ ]:
def deep_net_a():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(30,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(18, kernel_initializer='normal', activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.summary()

    
    return model

In [ ]:
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.decomposition import PCA

clf_a = Pipeline([
        ('scaler', MinMaxScaler()),
        ('feat_sel', SelectKBest(k=80, score_func=chi2)),
        ('feat_red', PCA(n_components=30)),
        ('classify', KerasClassifier(build_fn=deep_net_a, 
                           epochs=50, 
                           batch_size=35, 
                           validation_split=0.05)
        )
])

print(train_dy[train['h_target'] == 0].values.squeeze().shape)

clf_a.fit(train_dX[train['h_target'] == 0], train_dy[train['h_target'] == 0].values.squeeze())

### Interacting Material

In [ ]:
def deep_net_i():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(30,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, kernel_initializer='normal', activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.summary()

    
    return model

In [ ]:
clf_i = Pipeline([
        ('scaler', MinMaxScaler()),
        ('feat_sel', SelectKBest(k=80, score_func=chi2)),
        ('feat_red', PCA(n_components=30)),
        ('classify', KerasClassifier(build_fn=deep_net_i, 
                                              epochs=50, 
                                              batch_size=35,
                                              validation_split=0.15)
        )
])

clf_i.fit(train_dX[train['h_target'] == 1], train_dy[train['h_target'] == 1].values.squeeze())

### Scoring

In [ ]:
names_a = []
for name in np.unique(test_yy[test_y == 0]):
    names_a.append(classes[name])
names_i = []
for name in np.unique(test_yy[test_y == 1]):
    names_i.append(classes[name])
print(names_a)
print(names_i)

In [ ]:
pred = clf.predict(test_X)
print(metrics.accuracy_score(test_y, pred))
cm = metrics.confusion_matrix(test_y, pred)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, h_classes)
plt.show()

In [ ]:
pred = clf_a.predict(test_dX[test['h_target'] == 0])
print(metrics.accuracy_score(test_dy[test['h_target'] == 0].values.squeeze(), pred))
cm = metrics.confusion_matrix(test_dy[test['h_target'] == 0].values.squeeze(), pred)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, names_a)
plt.show()

In [ ]:
pred = clf_i.predict(test_dX[test['h_target'] == 1])
print(metrics.accuracy_score(test_dy[test['h_target'] == 1].values.squeeze(), pred))
cm = metrics.confusion_matrix(test_dy[test['h_target'] == 1].values.squeeze(), pred)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, names_i)
plt.show()

## Full Evaluation
We combine the classifiers to determine overall performance

In [ ]:
predictions = []
full_targets = []
start_time = time.time()
top_pred_probs = []
pred_probs = []

test = dataset[dataset.fold == 1].reset_index()
    
for data_iloc in range(0,len(test)):
    s_time = time.time()
    x_file_pp = ps.preprocess_file(test.iloc[data_iloc].filename,
                                                 blocksize=blocksize,
                                                 overlap=overlap)
    
    x_file = load_file_blockwise(test.iloc[data_iloc].filename, orig_blocksize, orig_overlap)
    
    y_file = test.iloc[data_iloc].h_target
    yy_file = test.iloc[data_iloc].target
    
    pred = clf.predict(x_file, verbose=0)[:,0]
    top_pred_probs.append(clf.predict_proba(x_file))
    prob = np.average(top_pred_probs[-1], axis=0)
    if prob[0] > 0.5:
        pred = clf_a.predict(x_file_pp, verbose=0)
        b = Counter(pred)
        predictions.append(b.most_common(1)[0][0])
        pred_probs.append(clf_a.predict_proba(x_file_pp))
    else:
        pred = clf_i.predict(x_file_pp, verbose=0)
        b = Counter(pred)
        predictions.append(b.most_common(1)[0][0])
        pred_probs.append(clf_i.predict_proba(x_file_pp))
    
    full_targets.append(yy_file)
    print("\tFile Time: " + str(time.time() - s_time))

print("\tProcessing Time: " + str(time.time() - start_time))



In [ ]:
print(metrics.accuracy_score(full_targets, predictions))
print(metrics.precision_score(full_targets, predictions, average='macro'))
cm = metrics.confusion_matrix(full_targets, predictions)
plt.figure(figsize=(20,20))
plot_confusion_matrix(cm, classes)
plt.show()

In [ ]:
def query_dataset(query_term):
    h_l = mapping[query_term]
    l_l = classes.index(query_term)
    
    predictions = []

    for data_iloc in range(0,len(test)):
        x_file_pp = ps.preprocess_file(test.iloc[data_iloc].filename,
                                                     blocksize=blocksize,
                                                     overlap=overlap)

        x_file = load_file_blockwise(test.iloc[data_iloc].filename, orig_blocksize, orig_overlap)

        y_file = test.iloc[data_iloc].h_target
        yy_file = test.iloc[data_iloc].target

        pred = clf.predict(x_file, verbose=0)[:,0]
        top_pred_probs.append(clf.predict_proba(x_file))
        prob = np.average(top_pred_probs[-1], axis=0)
        if prob[0] > 0.5:
            pred = clf_a.predict(x_file_pp, verbose=0)
            b = Counter(pred)
            predictions.append(b.most_common(1)[0][0])
            pred_probs.append(clf_a.predict_proba(x_file_pp))
        else:
            pred = clf_i.predict(x_file_pp, verbose=0)
            b = Counter(pred)
            predictions.append(b.most_common(1)[0][0])
            pred_probs.append(clf_i.predict_proba(x_file_pp))

        
        prob = clf.predict_proba(x_file)[l_l]
        pred = clf.predict(x_file[np.newaxis, :, :, :])
        
        predictions.append({
            'file': test.iloc[data_iloc].filename,
            'ds_id': data_iloc,
            'prob': prob,
            'prediction': pred
        })
        
    predictions = pd.DataFrame(predictions).sort_values(by=['prob'], ascending=False).reset_index(drop=True)
    return predictions